In [1]:
import sys
sys.path.insert(0, '../')

from src.time_tagger import TimeTaggerController, TT_Simulator, two_particle_states
from src.kinetic_mount_controller import KineticMountControl
from src.osc import OSCCommunicator, OSCTarget
import time
import threading
import random

# Setup Mount Controller

In [2]:
# only run if not already created
if 'KMC' not in globals():  
    KMC = KineticMountControl(number_of_devices=2, address_search_depth=0)
    #KMC.swap_alice_bob()


ASSIGNING CONTROLLERS
---------------------------------------------------------------
Ports Found: ['COM5', 'COM4']
---------------------------------------------------------------
Device succesfully connected (Rotation Mount) (Address: 0)
Device succesfully connected (Rotation Mount) (Address: 0)


In [1]:
# Init the Time Tagger and the Kinetic Mount Controller
TTC = TimeTaggerController()

# NOTE Look at the trace, sucessively block each port and assign the channels numbers in the next cell
# NOTE afterwards stop and close the graphic
#TTC.displayCountTraces()

NameError: name 'TimeTaggerController' is not defined

In [4]:
# connect KMC and TTC
TTC.setKineticMountController(KMC)
# set alice channels
TTC.set_alice_reflection_channel(4)
TTC.set_alice_transmission_channel(3)
# set bob channels
TTC.set_bob_reflection_channel(1)
TTC.set_bob_transmission_channel(2)


# set bob home offset
KMC.bob.set_home_offset(-43 % 360)
KMC.bob.home()
# set alice home offset
KMC.alice.set_home_offset(9)
KMC.alice.home()



In [5]:
_ = TTC.performDelayAdjustment(integration_time=5)#, manual_delays=[0, 98, -677, 2])

Delays Before Correction
Alice_T :     0 ps 	/     0.0 mm
Alice_R :   139 ps 	/    41.7 mm
Bob_T   :  -597 ps 	/  -179.1 mm
Bob_R   :    62 ps 	/    18.6 mm

Delays After Correction
Alice_T :     0 ps 	/     0.0 mm
Alice_R :    15 ps 	/     4.5 mm
Bob_T   :   -34 ps 	/   -10.2 mm
Bob_R   :   -13 ps 	/    -3.9 mm


In [46]:
KMC.rotate_simulataneously(45, 0)

In [19]:

KMC.rotate_simulataneously(0, 0)

In [9]:
KMC.rotate_simulataneously(45, 0)

In [18]:
n_values = 150
trace_width = 3
TTC.displayCountTraces(n_values=n_values, trace_width=trace_width, plot_title='<b>Raw Counts<b>')
TTC.displayCoincidenceTraces(coincidence_window_SI=0.5e-9, n_values=n_values, trace_width=trace_width, plot_title='<b>Coincidences<b>')

Output()

Output()

In [37]:
KMC.rotate_simulataneously(0, 0, wait_for_completion=False)

In [21]:

KMC.rotate_simulataneously(-22.5, -11.25, wait_for_completion=False)

In [ ]:
# TODO
# coincidence channel handling and destruction
#TTC.coincidences_vchannels.getChannels()

In [10]:
# TODO This kills the coincidence channels in the visualisation. Needs to be worked on
idealAngles = [0, -22.5, -11.25, 11.25] # for triplett |HH> - |VV> initial state
johannes_ideal_angles = [0, 45/2, -22.5/2, 22.5/2]
johannes_classical_angles = [0, 45/2, -90/2, -45/2]

In [19]:
TTC.measureS(CHSH_angles=idealAngles, integration_time_per_basis_setting_SI=2, TTSimulator=None, coincidence_window_SI=1.0e-9, debug=True)


corr[a,b] = 0.83001
	N[|T,T>]=  2777	(0.443) ███████████████████▉                         
	N[|T,R>]=   250	(0.040) █▊                                           
	N[|R,T>]=   283	(0.045) ██▏                                          
	N[|R,R>]=  2961	(0.472) █████████████████████▎                       

corr[a,B] = 0.51787
	N[|T,T>]=  2192	(0.363) ████████████████▍                            
	N[|T,R>]=   685	(0.113) █████▏                                       
	N[|R,T>]=   772	(0.128) █████▊                                       
	N[|R,R>]=  2395	(0.396) █████████████████▊                           

corr[A,b] = 0.33658
	N[|T,T>]=  1919	(0.302) █████████████▋                               
	N[|T,R>]=   875	(0.138) ██████▎                                      
	N[|R,T>]=  1235	(0.194) ████████▊                                    
	N[|R,R>]=  2332	(0.367) ████████████████▌                            

corr[A,B] = -0.63654
	N[|T,T>]=   476	(0.078) ███▌                                  

In [ ]:
TTC.measure_S_with_two_ports(CHSH_angles=idealAngles, debug=True)

SPCM Pairs: ['TT', 'TR', 'RT', 'RR']

corr[a,b] = [ 0.835 -0.81  -0.823  0.783]
	N[a , b ]=[1386  145  161 1222]
	N[a , b ]=[ 129 1183 1435  170]
	N[a , b ]=[ 106 1204 1251  112]
	N[a , b ]=[1231  106  100 1091]

corr[a,B] = [ 0.504 -0.488 -0.511  0.545]
	N[a , B ]=[1142  357  355 1153]
	N[a , B ]=[ 338 1065 1173  311]
	N[a , B ]=[ 376  972 1017  288]
	N[a , B ]=[1025  344  354  881]

corr[A,b] = [ 0.415 -0.377 -0.41   0.318]
	N[A , b ]=[999 399 357 924]
	N[A , b ]=[368 897 991 357]
	N[A , b ]=[ 449  925 1003  543]
	N[A , b ]=[975 425 477 814]

corr[A,B] = [-0.765  0.651  0.726 -0.631]
	N[A , B ]=[ 188 1103 1195  273]
	N[A , B ]=[1230  229  220  952]
	N[A , B ]=[1319  233  177 1156]
	N[A , B ]=[ 152 1083 1301  204]

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = TT, TR, RT, RR : [2.51871333 2.32588222 2.46946482 2.27627733]


In [9]:
#SimulatorPhiPlus = TT_Simulator(two_particle_states['phi_plus'], initial_state_noise=0.00, debug=False)
#SimulatorPhiMinus = TT_Simulator(two_particle_states['phi_minus'], initial_state_noise=0.00, debug=False)
#TTC.measureS(CHSH_angles=idealAngles, integration_time_per_basis_setting_SI=1, TTSimulator=SimulatorPhiPlus, coincidence_window_SI=0.5e-9)

# OSC Server Setup


In [20]:
my_ip = '192.168.0.2'
visual_ip = '192.168.0.5'
my_port = 7401 
clemens_port = 7401
visual_port = 7403

if 'OSC' not in globals():
    OSC = OSCCommunicator(my_ip=my_ip, my_port=my_port)

Myself = OSCTarget(ip=my_ip, port=my_port, name='Myself')
Clemens = OSCTarget(ip='192.168.0.3', port=clemens_port, name='Clemens')
Visuals = OSCTarget(ip=visual_ip, port=visual_port, name='Visuals')

# Start the server
OSC.start_server()

OSC Server Started. Serving on ('192.168.0.2', 7401)
----------------------------------------------------


In [21]:
# Send measurements
measurement_results = [1, 2, 3, 4]
OSC.send_measurement(target=Clemens, measurement_results=measurement_results)


Measurement [1, 2, 3, 4] sent to Clemens


In [11]:
OSC.send_visuals(target=Visuals, measurement_results="hi")

Measurement hi sent to Visuals


In [12]:
# Global flag to control the loop
performance_running = False
resultTranslate = {0 : [1, 1],
                   1 : [1, -1],
                   2 : [-1, 1],
                   3 : [-1, -1]}
# a A b B
Q_angles = [0, -22.5, -11.5, 11.5]
C_angles = [0, 22.5, -45, -22.5]

                    # ab, aB, Ab, AB
angle_pair_dict = {'Q_all':[(0, -11.5), (0, 11.5), (-22.5, -11.5), (-22.5, 11.5)],
                    # ab, AB
                   'Q_diagonal': [(0, 11.5), (-22.5, -11.5)],
                    # ab, AB
                   'C_diagonal': [(0, -45), (22.5, -22.5)],
                    # AB
                   'C_single': [(-45, -22.5)],
                   }

def performance_cycle():
    """The function you want to execute repeatedly."""
    global performance_running
    
    time_of_last_visual = time.time()
    while performance_running:
        theta_a, theta_b = random.choice(angle_pair_dict[OSC.currently_selected_state])
        
        measurement_results = TTC.get_single_measurement(theta_a, theta_b)
        
        # check if setup is measuring Quantum or Classical angles for conversion to clemens format        
        if OSC.currently_selected_state[0] == 'Q':
            check_angles = Q_angles
        elif OSC.currently_selected_state[0] == 'C':
            check_angles = C_angles

        resultClemensFormat = [1 if theta_a == check_angles[0] else 2, 1 if theta_b == check_angles[2] else 2 ,*resultTranslate[measurement_results]]

        OSC.send_measurement(Clemens, resultClemensFormat)

        #if time.time() - time_of_last_visual > 7:
        #    OSC.send_visuals(Visuals, resultClemensFormat)
        #    time_of_last_visual = time.time()


def start_performance():
    """Starts the task in a separate thread."""
    global performance_running
    performance_running = True  # Ensure the loop is enabled
    task_thread = threading.Thread(target=performance_cycle, daemon=True)
    task_thread.start()

def stop_performance():
    """Signals the task to stop gracefully."""
    global performance_running
    performance_running = False
    print('performance stopped...')


In [13]:
Myself.client.send_message("/bruQner/set/state", 'Q_all')

In [15]:
stop_performance()

performance stopped...


In [14]:
start_performance()

Collected 154 events in 0.10 seconds.
Measurement [2, 2, 1, -1] sent to Clemens
Collected 114 events in 0.10 seconds.
Measurement [2, 2, 1, 1] sent to Clemens
Collected 177 events in 0.10 seconds.
Measurement [1, 1, -1, -1] sent to Clemens
Collected 157 events in 0.10 seconds.
Measurement [2, 1, 1, 1] sent to Clemens
Collected 168 events in 0.10 seconds.
Measurement [1, 1, -1, 1] sent to Clemens
Collected 190 events in 0.10 seconds.
Measurement [2, 1, -1, -1] sent to Clemens
Collected 146 events in 0.10 seconds.
Measurement [1, 2, 1, 1] sent to Clemens
Collected 170 events in 0.10 seconds.
Measurement [1, 2, -1, -1] sent to Clemens
Collected 197 events in 0.10 seconds.
Measurement [2, 2, -1, 1] sent to Clemens
Collected 194 events in 0.10 seconds.
Measurement [2, 1, -1, 1] sent to Clemens
Collected 203 events in 0.10 seconds.
Measurement [2, 1, -1, 1] sent to Clemens
Collected 198 events in 0.10 seconds.
Measurement [2, 1, 1, 1] sent to Clemens
Collected 203 events in 0.10 seconds.
Mea